In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
%%time
import json
import pandas as pd
import numpy as np
import cv2
from PIL import Image
#from tesserocr import PyTessBaseAPI, PSM, RIL, PT
import matplotlib.pyplot as plt

CPU times: user 587 ms, sys: 313 ms, total: 900 ms
Wall time: 10.8 s


In [3]:
%%time
import lib.utils as utils
import lib.config as config
import lib.model as model
import lib.roi as roi
import lib.split_pages as spages
import lib.utils as utils

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 1.9 s, sys: 1.09 s, total: 2.99 s
Wall time: 31.3 s


In [4]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg"
image_dir = "images/cropped/"

In [5]:
images = sorted(utils.load_images(image_dir))[:2]

In [6]:
newprompt = """
The input image contains a cropped image of a botanical catalogue listing the contents of folders sorted and listed in taxonomic order.
These information are splitted into columns in each page.
The image provides the division name, family name, species name and their folders' contents. 
---
An example of the provided format is:

Dicotyledones
ACERACEAE
Acer campestre L.
1 folder. Acer campestre [TA]
Acer pseudoplatanus L.
2 folders.
Folder 1. Acer Pseudo-Platanus
[G]. i. "Maple. Bulls: [Bulstrode]
Park" [JL]
Folder 2. Acer Pseudo-Platanus
[TA].
AMARANTHACEAE
Amaranthus lividus L. Flora Europaea 1: 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cities Ray's Syn. 1957. ii. "Blite 
Amaranth. Aug. It is often found
on Dunghills in the neighbourhood
of London. I gather'd this on a
Dunghill at Fulham near London"
[JL]. iii. "Amaranthus Blitum.
Monoec: 5. and:" [JL]
---

where divison names are characterised by large bold words; family names as captial words; species names as italic words.
The list of contents under each species lists the number of folders first and then their content. 
When more than one folder is provided roman (like i, ii, iii) are used to differentiate the folders.
Furthermore, for each collection the collectors initials are also cited using square brackets '[]'.

In some cases, the input image provided contains folder information from previously provided images and thus these information had continued from the last page.

---
Example of output lists of JSON dicts

[
{
'division':'Dicotyledones'
'contents': [{'family'='ACERACEAE', contents=[{'species'='Acer campestre L.', folders=[{'folder'=1, 'content'='Acer campestre [TA]'}]}, 
                                              {'species'='Acer pseudoplatanus L.', folders=[{'folder'=1, 'content'='Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]'},
                                              {'folder'=2, 'content'='Acer Pseudo-Platanus [TA]'}]}]
},{'family'='AMARANTHACEAE', contents=[{'species'='Amaranthus lividus L. Flora Europaea 1: 110 (1964)', folders=[{'folder'=1, 'content'='Amaranthus Blitum [TA].
i. Cities Ray's Syn. 1957. ii. "Blite 
Amaranth. Aug. It is often found
on Dunghills in the neighbourhood
of London. I gather'd this on a
Dunghill at Fulham near London"
[JL]. iii. "Amaranthus Blitum.
Monoec: 5. and:" [JL]'}]}, ]}

]}

---


Process the image using the following steps to generate a JSON file that supports the above provided structure

1. Create a list of JSON files
2. Find the division name where possible, if not available set as 'N/A'.
3. Under the division, create a contents lists
4. For each family name under division name create a JSON dict and add it to the list
5. Under each family name find the species name and thier folder contents and add them to the list
6. Where appropriate if a value for a specific key cannot be found, set it as 'N/A'
7. In cases where extra text is provided before Division name, family names or species name, consider the division, family or species name as 'Extra info'

Compile these information into a list of JSON dicts and return the list. Please output this list of JSON dicts and not the example. 
The example (denoted between --- and ---) is only to be used as a template for the structure of the JSON format.


Family names are characterised by captial words whilst species names are characterised by italic words.
The list of contents under each species lists the number of folders first and then their content. 
Furthermore, for each collection the collectors initials are also cited using square brackets '[]'.
"""

In [7]:
simple_prompt = """
The input image contains a cropped image of a botanical catalogue listing the contents of folders sorted and listed in taxonomic order.
The image provides the division name, family name, species name and their folders' contents. 

Transcribe the text in the image into a structured and labelled format.
"""

In [8]:
%%time
qwen_model = model.QWEN_Model(prompt=simple_prompt, max_new_tokens = 5000, batch_size=1)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:58<00:00, 11.72s/it]


CPU times: user 2.89 s, sys: 11.2 s, total: 14 s
Wall time: 1min 1s


In [9]:
%%time
outputs = qwen_model.batch_infer(images, save=False, debug=True)

>>> Using: 
 	Maximum new tokens = 5000 
 	Batch size = 1 
 	save_path = None
>>> Batch 1 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
>>> Batch 2 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
CPU times: user 1min 33s, sys: 3.73 s, total: 1min 37s
Wall time: 1min 46s


In [10]:
outputs

[('images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg',
  '```json\n{\n  "division": "Dicotyledones",\n  "family": "Aceraceae",\n  "species": "Acer campestre L.",\n  "folders": [\n    {\n      "folder": "1",\n      "contents": "Acer campestre [TA]"\n    }\n  ],\n  "species": "Acer pseudoplatanus L.",\n  "folders": [\n    {\n      "folder": "1",\n      "contents": "Acer Pseudo-Platanus [G]. i. \\"Maple. Bulls: [Bulstrode] Park\\" [JL]"\n    },\n    {\n      "folder": "2",\n      "contents": "Acer Pseudo-Platanus [TA]"\n    }\n  ],\n  "family": "Amaranthaceae",\n  "species": "Amaranthus lividus L., Flora Europaea",\n  "folders": [\n    {\n      "folder": "1",\n      "contents": "Amaranthus Blitum [TA]. i. Cites Ray\'s Syn. 1957. ii. \\"Blite Amaranth. Aug.\\" It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London\\" [JL]. iii. \\"Amaranthus Blitum. Monoec: 5. and:\\" [JL]"\n    }\n  ],\n  "family": "Araliaceae",\n  "specie

In [15]:
out = outputs[1][1]

In [16]:
print(out)

```json
{
  "division": "Caprifoliaceae",
  "family": "Linnaea borealis L.",
  "species": "Linnaea borealis",
  "folders": [
    {
      "folder": "1 sheet. Linnaea borealis [JL].",
      "contents": "Cites Linn. Sp. Pl. 631; Bauh. Pin. 93. ii. \"This is a Native of Sweden, Siberia, Switzerland, Russia, Canada, & c. This Specimen was gathered & communicated by Jos: Banks Esqr. who discover’d it in Abundance in the Woods in Newfoundland\" [JL]. iii. \"In Scotland—See English Botany N.433\" [? G]."
    }
  ]
}
```
